In [12]:
import pandas as pd
import numpy as  np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from IPython.display import display,HTML

In [13]:
data_br = load_breast_cancer()
X_br = data_br.data
Y_br = data_br.target

data_ab = pd.read_csv('abalone.csv')
X_ab = data_ab.iloc[:,:-1]
X_ab = pd.get_dummies(X_ab)
Y_ab = data_ab['Rings']
X_ab = np.array(X_ab)
Y_ab = np.array(Y_ab)

In [14]:
def calc_acc(X,Y,classifier,tsize,iteration):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=tsize,  random_state=3)
    accuracy = []
    for i in range(iteration):
        classifier.fit(X_train, Y_train)
        Y_pred = classifier.predict(X_test)
        accuracy.append(metrics.accuracy_score(Y_pred,Y_test)*100)
    accuracy = np.array(accuracy)
    accuracy = np.mean(accuracy)
    return accuracy

def calc_cross_val(X,Y,classifier,k):
    from sklearn.model_selection import cross_validate,cross_val_score
    score = cross_val_score(classifier, X, Y, cv=k)
    acc = (score*100).mean()
    return acc

In [15]:
classifiers = {'Naive_bayes':GaussianNB()  , 'KNN':KNeighborsClassifier(n_neighbors=5) , 'Decision_Tree':DecisionTreeClassifier()}
test_size = {'test_size= 0.25': 0.25 , 'test_size=0.33': 0.33}
data = {'Abalone':[X_ab,Y_ab] , 'Breast_Cancer':[X_br,Y_br]}
folds={'folds=4':4,'folds=5':5}
df = pd.DataFrame(np.empty((3,4)),columns=pd.MultiIndex.from_product([test_size.keys(),data.keys()]),index=classifiers.keys())       #HoldOut
df10 = pd.DataFrame(np.empty((3,4)),columns=pd.MultiIndex.from_product([test_size.keys(),data.keys()]),index=classifiers.keys())    #RandomSubsampling
dfcross = pd.DataFrame(np.empty((3,4)),columns=pd.MultiIndex.from_product([folds.keys(),data.keys()]),index=classifiers.keys())   #CrossValidation

for t_key,t_value in test_size.items():
    for d_key,d_value in data.items():
        for c_key,c_value in classifiers.items():
            df.loc[c_key,(t_key,d_key)]= calc_acc(d_value[0],d_value[1],c_value,t_value,1)
            df10.loc[c_key,(t_key,d_key)]= calc_acc(d_value[0],d_value[1],c_value,t_value,10)
            
for f_key,f_value in folds.items():
    for d_key,d_value in data.items():
        for c_key,c_value in classifiers.items():
            dfcross.loc[c_key,(f_key,d_key)]= calc_cross_val(d_value[0],d_value[1],c_value,f_value)

C:\Users\vipof\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)
C:\Users\vipof\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)
C:\Users\vipof\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)
C:\Users\vipof\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The mi

In [16]:
display(HTML(df.to_html()))

In [17]:
display(HTML(df10.to_html()))

In [18]:
display(dfcross)

folds=4                  folds=5              
                 Abalone Breast_Cancer    Abalone Breast_Cancer
Naive_bayes    10.205745     94.204915  10.163534     94.037707
KNN            22.927522     92.798926  23.767825     92.804925
Decision_Tree  18.564217     92.448045  18.493781     91.916891